In [10]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import tokenizer_from_json
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Dropout
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.models import model_from_json
from keras.models import Model
from matplotlib import pyplot as plt

from keras.utils import pad_sequences
import os

def homePath(path):
    return os.path.join(os.path.expanduser("~"), path.strip("~/"))



#Model Creation 1
#Figure out how to do versioning effectively
individualInput = False
combinedInputPath = homePath("~/SPGenerations/")

GradesTokensName = ""
if individualInput:
    pathToTokenizedData = "C:\\Users\\mcall\\OneDrive\\Desktop\\DummyOutput\\Tokenizer\\"
    pathToGradeData = "C:\\Users\\mcall\\OneDrive\\Desktop\\DummyOutput\\Grader\\"
else:
    pathToTokenizedData = os.path.join(combinedInputPath, "Tokens/")
    pathToGradeData = os.path.join(combinedInputPath, "Grades/")
if GradesTokensName == "":
    #Use newest folder for each

    #Get the newest folder for the tokens
    tokensFolders = os.listdir(pathToTokenizedData)
    tokensFolders.sort()
    pathToTokenizedData = os.path.join(pathToTokenizedData , tokensFolders[-1])

    #Get the newest folder for the grades
    gradesFolders = os.listdir(pathToGradeData)
    gradesFolders.sort()
    pathToGradeData = os.path.join(pathToGradeData,  gradesFolders[-1])
else:
    pathToTokenizedData = os.path.join(pathToTokenizedData, GradesTokensName)
    pathToGradeData = os.path.join(pathToGradeData, GradesTokensName)





modelOutputPath = homePath("~/SPGenerations/Models")

#Load the data
#tokenized data is in tokenizedData.pkl, has tokenizer obj in tokenizer.json

#Load the tokenizer
with open(pathToTokenizedData+ "/tokenizer.json", "r") as f:
    tokenizer = tokenizer_from_json(f.read())



#Load the tokenized data
with open(pathToTokenizedData + "/tokenizedData.pkl", "rb") as f:
    tokenizedData = pickle.load(f)

#Load the grade data
#gradeData is a dict with keys as the file names and values as the grades
with open(pathToGradeData + "/grades.pkl", "rb") as f:
    gradeData = pickle.load(f)










['1697233160']


In [14]:
#combine the dataframes using the file path

#Join the dataframes
combinedDF = pd.merge(tokenizedData, gradeData, on = "Path")
print (combinedDF) 

#create a dataframe of grades that did not match with anything
#mismatchDF = gradeData[~gradeData.Path.isin(combinedDF.Path)]
#print (len(mismatchDF))
#print (mismatchDF)

#create a dataframe of files that did not match with anything
#mismatchDF2 = tokenizedData[~tokenizedData.Path.isin(combinedDF.Path)]
#print (len(mismatchDF2))



                                             tokenCode  \
0    [38, 53, 2, 38, 41, 2, 52, 123, 3, 161, 3, 104...   
1    [38, 53, 2, 38, 41, 2, 52, 123, 3, 161, 3, 104...   
2    [38, 53, 2, 38, 509, 2, 52, 68, 3, 890, 38, 36...   
3    [38, 53, 2, 38, 41, 2, 52, 68, 38, 75, 2, 52, ...   
4    [38, 53, 2, 38, 509, 2, 52, 68, 3, 890, 38, 36...   
..                                                 ...   
877  [38, 405, 2, 38, 959, 2, 38, 633, 2, 38, 828, ...   
878  [38, 828, 2, 38, 633, 2, 38, 405, 2, 38, 959, ...   
879  [38, 828, 2, 38, 633, 2, 38, 405, 2, 38, 959, ...   
880  [38, 828, 2, 38, 633, 2, 38, 405, 2, 38, 959, ...   
881  [38, 828, 2, 38, 633, 2, 38, 405, 2, 38, 959, ...   

                                      Path  fileGrade  
0                /3584343/conftest.py/1.py  50.000000  
1                /3584343/conftest.py/0.py  40.000000  
2       /3584343/tests/test_engine.py/5.py  47.692308  
3    /3584343/tests/test_graphson2.py/0.py  40.000000  
4       /3584343/tests/

In [ ]:
#create a chart of the grades
graph = plt.hist(dataSet1['grade'], bins=20, range=(0, 100))
plt.xticks([0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100])
plt.xlabel("Grade Range")
plt.ylabel("Number of Commits")
plt.show()

print(len(dataSet1temp))